In [1]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
import time

In [2]:
# Chargement des données depuis le fichier CSV avec l'encodage approprié
data = pd.read_csv('ecommerce-data/data.csv', encoding='ISO-8859-1')

In [3]:
# Calcul de la colonne 'GroupPrice'
data['GroupPrice'] = data['Quantity'] * data['UnitPrice']

# Affichage des dimensions du jeu de données
print('Dimensions du jeu de données : ', data.shape)

Dimensions du jeu de données :  (541909, 9)


In [4]:
# Suppression des lignes contenant des valeurs manquantes
data = data.dropna()

# Affichage des nouvelles dimensions après la suppression des valeurs manquantes
print('Les nouvelles dimensions après la suppression des valeurs manquantes :', data.shape)
print('---------')
print(data.head())

Les nouvelles dimensions après la suppression des valeurs manquantes : (406829, 9)
---------
  InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

      InvoiceDate  UnitPrice  CustomerID         Country  GroupPrice  
0  12/1/2010 8:26       2.55     17850.0  United Kingdom       15.30  
1  12/1/2010 8:26       3.39     17850.0  United Kingdom       20.34  
2  12/1/2010 8:26       2.75     17850.0  United Kingdom       22.00  
3  12/1/2010 8:26       3.39     17850.0  United Kingdom       20.34  
4  12/1/2010 8:26       3.39     17850.0  United Kingdom       20.34  


In [5]:
# Liste des codes de stock uniques
liste = data['StockCode'].unique() 

# Création d'une liste de codes de stock à supprimer
stock_to_del = []
for el in liste:
    # Supprimer les produits correspondant à des cadeaux (codes de stock qui ne commencent pas par un chiffre)
    if el[0] not in ['1','2','3','4','5','6','7','8','9','10']:
        stock_to_del.append(el)

# Suppression des produits indésirables du jeu de données
data = data[data['StockCode'].map(lambda x: x not in stock_to_del)]

In [6]:
# Grouper les produits par facture et client
basket = data.groupby(['InvoiceNo', 'CustomerID'])['StockCode'].apply(list).reset_index(name='StockCodeList')

# Affichage des dimensions du nouveau jeu de données regroupé
print('Dimension du nouveau jeu de données regroupé :', basket.shape)
print('----------')
print(basket.head())

Dimension du nouveau jeu de données regroupé : (21788, 3)
----------
  InvoiceNo  CustomerID                                      StockCodeList
0    536365     17850.0  [85123A, 71053, 84406B, 84029G, 84029E, 22752,...
1    536366     17850.0                                     [22633, 22632]
2    536367     13047.0  [84879, 22745, 22748, 22749, 22310, 84969, 226...
3    536368     13047.0                       [22960, 22913, 22912, 22914]
4    536369     13047.0                                            [21756]


In [7]:
# Convertir le format pour l'Apriori
te = TransactionEncoder()
basket_encoded = te.fit_transform(basket['StockCodeList'].apply(lambda x: [str(i) for i in x]))

In [8]:
# Convertir en DataFrame
basket_df = pd.DataFrame(basket_encoded, columns=te.columns_)

In [9]:
# Appliquer l'algorithme Apriori avec le support minimal et la confiance minimale spécifiés
min_support_threshold = 0.005
min_confidence_threshold = 0.5

In [10]:
# Mesure du temps d'exécution
a = time.time()

# Application de l'algorithme apriori pour extraire les fréquent items sets et les règles d'association
frequent_itemsets_apriori = apriori(basket_df, min_support=min_support_threshold, use_colnames=True)

# Mesure du temps d'exécution
b = time.time()

# Affichage du temps d'exécution en secondes
print("Temps d'exécution en secondes : ", b - a, 's.')

Temps d'exécution en secondes :  1086.1489026546478 s.


In [11]:
# Générer les règles d'association avec la confiance minimale
rules_apriori = association_rules(frequent_itemsets_apriori, metric="confidence", min_threshold=min_confidence_threshold)

In [12]:
# Afficher les ensembles fréquents de l'Apriori
print('Les ensembles fréquents de trouvé par Apriori:')
print(frequent_itemsets_apriori)

Les ensembles fréquents de trouvé par Apriori:
       support                                    itemsets
0     0.005645                                     (10133)
1     0.005416                                     (10135)
2     0.015054                                     (15036)
3     0.013585                                   (15056BL)
4     0.017624                                    (15056N)
...        ...                                         ...
3159  0.006334         (22920, 22916, 22918, 22921, 22917)
3160  0.006150         (22920, 22919, 22916, 22921, 22917)
3161  0.006196         (22920, 22919, 22916, 22918, 22921)
3162  0.006196         (22920, 22919, 22918, 22921, 22917)
3163  0.006104  (22920, 22919, 22916, 22918, 22921, 22917)

[3164 rows x 2 columns]


In [13]:
# Appliquer l'algorithme Apriori avec des paramètres de support minimal = 0.01 et confiance minimal = 0.8
min_support_threshold = 0.01
min_confidence_threshold = 0.8

In [14]:
# Mesure du temps d'exécution
a = time.time()

frequent_itemsets_apriori = apriori(basket_df, min_support=min_support_threshold, use_colnames=True)


# Mesure du temps d'exécution
b = time.time()

# Affichage du temps d'exécution en secondes
print("Temps d'exécution en secondes : ", b - a, 's.')

Temps d'exécution en secondes :  77.07467651367188 s.


In [15]:
# Générer les règles d'association avec la confiance minimale
rules_apriori = association_rules(frequent_itemsets_apriori, metric="confidence", min_threshold=min_confidence_threshold)

In [17]:
# Afficher les règles d'association avec le support 0.01 et la confiance 0.8
rules_apriori_with_metrics = rules_apriori[['antecedents', 'consequents', 'support', 'confidence']]
print("Les ", len(rules_apriori_with_metrics), " premières règles d'association:")
print(rules_apriori_with_metrics.sort_values(by='confidence', ascending=False))

Les  10  premières règles d'association:
             antecedents consequents   support  confidence
8  (22699, 22698, 22423)     (22697)  0.011061    0.889299
6         (22699, 22698)     (22697)  0.018221    0.880266
9  (22423, 22698, 22697)     (22699)  0.011061    0.879562
3         (22698, 22423)     (22697)  0.012576    0.858934
5         (22698, 22423)     (22699)  0.012438    0.849530
2                (23171)     (23170)  0.010648    0.843636
7         (22698, 22697)     (22699)  0.018221    0.842887
0                (21086)     (21094)  0.010832    0.828070
4         (22423, 22697)     (22699)  0.014503    0.825065
1                (22579)     (22578)  0.010419    0.819495
